In [2]:
import pandas as pd
%load_ext rpy2.ipython

/opt/anaconda/lib/python2.7/site-packages/pytz/__init__.py:29: UserWarning: Module argparse was already imported from /opt/anaconda/lib/python2.7/argparse.pyc, but /opt/anaconda/lib/python2.7/site-packages is being added to sys.path
  from pkg_resources import resource_stream


In [11]:
%%R
library(data.table)
library(dplyr); library(tidyr)
library(ggplot2)

In [6]:
!expr $(cat ../data/ABCD_combined_reads.fastq | wc -l) / 4

1720480


In [5]:
!grep -c ">" ../data/finalQC.fasta

1376008


In [44]:
%%R
mdf = tbl_df(fread("../data/mdf.csv"))

Read 2195072 rows and 18 (of 18) columns from 0.435 GB file in 00:00:03


In [17]:
%%R
mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS")) %>%
    group_by(Sample) %>%
    summarize(N.seqs = sum(Abundance)) %>%
    group_by() %>%
    summarize(mean(N.seqs), sd(N.seqs), min(N.seqs), max(N.seqs), sum(N.seqs >= 1000), N = n())

Source: local data frame [1 x 6]

  mean(N.seqs) sd(N.seqs) min(N.seqs) max(N.seqs) sum(N.seqs >= 1000)   N
1     3815.519   3629.896         222       41583                 265 289


In [67]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"))

bulkIDs = grep("bulk", unique(d$Sample), value = TRUE)
length(bulkIDs)

[1] 12


In [66]:
%%R
mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), !(Sample %in% bulkIDs)) %>%
    group_by(Day, Treatment) %>%
    summarize(N = length(unique(Sample))) %>%
    group_by() %>%
    summarize(mean(N), sd(N), sum(N))

Source: local data frame [1 x 3]

   mean(N)     sd(N) sum(N)
1 19.78571 0.5789342    277


In [50]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), Abundance > 0)

length(unique(d$OTU))

[1] 5940


In [49]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), Abundance > 0, Sample %in% bulkIDs)

length(unique(d$OTU))

[1] 2943


In [54]:
%%R
mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), !(Sample %in% bulkIDs)) %>%
    group_by(Day, Treatment) %>%
    summarize(avg.den = (max(Density) - min(Density)) / length(unique(Sample))) %>%
    group_by() %>%
    summarize(mean(avg.den))

Source: local data frame [1 x 1]

  mean(avg.den)
1   0.004027387


In [63]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), Abundance > 0)

length(na.omit(unique(d$Rank2)))

[1] 33


In [64]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), Abundance > 0)

length(na.omit(unique(d$Rank6)))

[1] 340


In [65]:
%%R
d = mdf %>%
    filter(Treatment %in% c("12CCPS", "13CCPS", "13CXPS"), !(Sample %in% bulkIDs)) 

length(unique(with(d, paste(Day, Treatment))))

[1] 14
